# Evaluate Kahoot performance during a course

## Evaluate points

### Read data from Excell files

In [1]:
import os
import pandas

directory_name = "A:\\1docs\\aktualis\\2017-18-2 Oktatas\\Fusion devices\\kahoot"
sheet_name = "Final Scores"
list_of_all_files = os.listdir(directory_name)
list_of_all_files

['2results-1518439565657-b991d32d-a9b6-4002-bead-2429323a341c.xlsx',
 '4results-1519648851208-81bec4cc-d083-43ec-932c-2b07d210312a.xlsx',
 '3results-1519643740373-700a838d-54a3-4de8-a51b-29684c3c9c2a.xlsx',
 '5results-1520253618743-dd3ce2c9-f76f-4c26-ae4f-55e9892fcf2e.xlsx',
 '4Rresults-1520248388759-b6dfaac6-2e3c-45d9-ac52-740f539e3433.xlsx',
 '6results-1520858117269-74b88c16-404a-4a7f-955d-e53d2f017880.xlsx',
 '5Rresults-1520852958909-2c51596f-dcd1-470f-b8ac-480f06624bdb.xlsx',
 '6Rresults-1521457653666-05abe9d0-d7d3-44cb-8d8e-7001c8ab4979.xlsx',
 '7results-1521463155780-422d502a-84b1-4dca-8fad-ef25690f5e30.xlsx',
 '9Vresults-1525693507486-51ca6493-5aee-4945-83fe-4407cb6af716.xlsx',
 'kahoot_results.xlsx']

In [15]:
list_of_files = [elem for elem in list_of_all_files if len(elem) > 20]
list_of_files

['2results-1518439565657-b991d32d-a9b6-4002-bead-2429323a341c.xlsx',
 '4results-1519648851208-81bec4cc-d083-43ec-932c-2b07d210312a.xlsx',
 '3results-1519643740373-700a838d-54a3-4de8-a51b-29684c3c9c2a.xlsx',
 '5results-1520253618743-dd3ce2c9-f76f-4c26-ae4f-55e9892fcf2e.xlsx',
 '4Rresults-1520248388759-b6dfaac6-2e3c-45d9-ac52-740f539e3433.xlsx',
 '6results-1520858117269-74b88c16-404a-4a7f-955d-e53d2f017880.xlsx',
 '5Rresults-1520852958909-2c51596f-dcd1-470f-b8ac-480f06624bdb.xlsx',
 '6Rresults-1521457653666-05abe9d0-d7d3-44cb-8d8e-7001c8ab4979.xlsx',
 '7results-1521463155780-422d502a-84b1-4dca-8fad-ef25690f5e30.xlsx',
 '9Vresults-1525693507486-51ca6493-5aee-4945-83fe-4407cb6af716.xlsx']

### Players corrections

In [3]:
import numpy as np
def convert_players(old_player):
    if old_player == "Niklas":
        return "TJBYX4"
    if old_player == "Vladislav M.":
        return "DHD5TF"
    if old_player == "Bori":
        return "KD9XVQ"
    if old_player == "Yossi":
        return "SW0S1F"
    if old_player == "Tobi":
        return "TH5Z6J"
    if old_player == "Buchtinator":
        return "F87N7B"
    if old_player == "Daniel Brunsch":
        return "MU3HCF"
    if old_player == "Vicky":
        return "WE2600"
    if old_player == "H.Adam":
        return "PJ257S"
    if old_player == "Mo":
        return "RYLBNG"
    old_player = str(old_player).upper()
    if old_player == "MUH3CF":
        return "MU3HCF"
    return old_player

conv_players=np.vectorize(convert_players)

### Read data from Excell files and perform transformations

In [4]:
scores = pandas.DataFrame()
for file_name in list_of_files:
    scores_single = pandas.read_excel(directory_name+"\\"+file_name, sheetname=sheet_name, header=1, 
                                      skiprows=[1], skipfooter=1,
                                      converters={"Players":convert_players})
    scores = scores.append(scores_single)

In [5]:
scores_single.head()

,Rank,Players,Total Score (points),Correct Answers,Incorrect Answers
0,1.0,SW0S1F,3530.0,4.0,1.0
1,2.0,AO4HT7,3386.0,4.0,1.0
2,3.0,F87N7B,2609.0,3.0,2.0
3,4.0,WE2600,2225.0,3.0,1.0
4,5.0,TH5Z6J,2139.0,3.0,0.0


In [6]:
scores.head()

,Rank,Players,Total Score (points),Correct Answers,Incorrect Answers
0,1.0,TJBYX4,3934.0,6.0,2.0
1,2.0,DHD5TF,3350.0,6.0,2.0
2,3.0,KD9XVQ,3344.0,6.0,1.0
3,4.0,SW0S1F,2748.0,5.0,2.0
4,5.0,TH5Z6J,2722.0,5.0,3.0


In [7]:
scores['Players'].value_counts()

TJBYX4    10
F87N7B    10
MU3HCF    10
RYLBNG    10
KD9XVQ    10
TH5Z6J    10
DHD5TF    10
BXC03A     9
SW0S1F     8
WE2600     8
SSHHGP     7
PJ257S     3
AO4HT7     1
Name: Players, dtype: int64

### Aggregate results of individual Players

In [8]:
scores_aggregate = scores.groupby('Players',as_index=False).agg({'Rank': ['mean', 'count'],
                                                                'Total Score (points)': 'sum',
                                                                'Correct Answers': 'sum',
                                                                'Incorrect Answers': 'sum'})

In [9]:
scores.columns

Index(['Rank', 'Players', 'Total Score (points)', 'Correct Answers',
       'Incorrect Answers'],
      dtype='object')

In [10]:
scores_aggregate.columns=[ 'Players', 'Rank', 'Count', 'Total Score (points)', 'Correct Answers', 'Incorrect Answers']


In [11]:
scores_aggregate

,Players,Rank,Count,Total Score (points),Correct Answers,Incorrect Answers
0,AO4HT7,2.000000,1,3386.0,4.0,1.0
1,BXC03A,8.444444,9,14677.0,19.0,18.0
2,DHD5TF,5.100000,10,24415.0,30.0,13.0
3,F87N7B,4.400000,10,26437.0,29.0,22.0
4,KD9XVQ,6.500000,10,21913.0,28.0,10.0
5,MU3HCF,5.600000,10,23498.0,31.0,19.0
6,PJ257S,4.666667,3,10429.0,12.0,6.0
7,RYLBNG,6.600000,10,21346.0,28.0,15.0
8,SSHHGP,9.285714,7,9358.0,12.0,13.0
9,SW0S1F,3.125000,8,26166.0,30.0,10.0


In [12]:
scores_aggregate['Total Score (Percentage)'] = 100.0 * scores_aggregate['Total Score (points)'] / \
                                               scores_aggregate['Total Score (points)'].max()

In [13]:
scores_aggregate

,Players,Rank,Count,Total Score (points),Correct Answers,Incorrect Answers,Total Score (Percentage)
0,AO4HT7,2.000000,1,3386.0,4.0,1.0,12.807807
1,BXC03A,8.444444,9,14677.0,19.0,18.0,55.516889
2,DHD5TF,5.100000,10,24415.0,30.0,13.0,92.351628
3,F87N7B,4.400000,10,26437.0,29.0,22.0,100.000000
4,KD9XVQ,6.500000,10,21913.0,28.0,10.0,82.887620
5,MU3HCF,5.600000,10,23498.0,31.0,19.0,88.883005
6,PJ257S,4.666667,3,10429.0,12.0,6.0,39.448500
7,RYLBNG,6.600000,10,21346.0,28.0,15.0,80.742898
8,SSHHGP,9.285714,7,9358.0,12.0,13.0,35.397360
9,SW0S1F,3.125000,8,26166.0,30.0,10.0,98.974922


### Print result

In [21]:
writer = pandas.ExcelWriter(directory_name+"\\"+'kahoot_results.xlsx')
scores_aggregate.to_excel(writer,'Results')
writer.save()

## Evaluate question statistics